In [1]:
import pandas as pd
from plotnine.data import mtcars

import numpy as np

from sklearn.preprocessing import StandardScaler

import statistics

import itertools

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [6]:
#fonte = mtcars[['mpg','disp']].copy()

fonte = mtcars[['mpg','cyl']].copy()
fonte = fonte.rename({'cyl': 'disp'}, axis = 1)
fonte['disp'] = fonte['disp'].astype(float)

quantidade_linhas = 7
fonte = fonte.loc[0:quantidade_linhas-1]
display(fonte)
display(len(fonte))

,mpg,disp
0,21.0,6.0
1,21.0,6.0
2,22.8,4.0
3,21.4,6.0
4,18.7,8.0
5,18.1,6.0
6,14.3,8.0


7

In [11]:
print(fonte['mpg'].mean())
print(fonte['disp'].mean())

19.614285714285717
6.285714285714286


In [10]:
print(fonte['mpg'].std())
print(fonte['disp'].std())

2.8445436490636897
1.3801311186847083


In [3]:
lista_index = []

for i in range(1, quantidade_linhas+1):
    if i < 10:
        lista_index.append("a0"+str(i))
    else:
        lista_index.append("a"+str(i))
    
print(lista_index)

fonte.index = lista_index

['a01', 'a02', 'a03', 'a04', 'a05', 'a06', 'a07']


In [4]:
for coluna in fonte.columns:
    media = round(fonte[coluna].mean(), 2)
    desvio_padrao = round(fonte[coluna].std(), 2)
    for i in range(0, len(fonte)):
        fonte[coluna][i] = (fonte[coluna][i] - media)/desvio_padrao
        
fonte = fonte

In [5]:
fonte

6.29

In [5]:
fonte

,mpg,disp
a01,0.489437,-0.210145
a02,0.489437,-0.210145
a03,1.123239,-1.659420
a04,0.630282,-0.210145
a05,-0.320423,1.239130
a06,-0.531690,-0.210145
a07,-1.869718,1.239130


In [6]:
df = pd.DataFrame(index=fonte.index, columns = list(fonte.index))
df

,a01,a02,a03,a04,a05,a06,a07
a01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a07,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
for coluna in df.columns:
    #print(coluna) #cima
    for linha in df.index:
        #print(linha) #baixo
        
        mpg = (fonte['mpg'][coluna] - fonte['mpg'][linha])**2 
        mpg = round(mpg, 2)
        
        disp = (fonte['disp'][coluna] - fonte['disp'][linha])**2
        disp = round(disp, 2)
        
        valor = mpg + disp
        valor = round(valor, 2)
        
        valor = valor/2
        valor = round(valor, 2)
        
        valor = valor**0.5
        valor = round(valor, 2)
        
        
        df[coluna][linha] = valor
        
df_completo = df.copy()
df

,a01,a02,a03,a04,a05,a06,a07
a01,0.0,0.0,1.12,0.1,1.17,0.72,1.96
a02,0.0,0.0,1.12,0.1,1.17,0.72,1.96
a03,1.12,1.12,0.0,1.08,2.29,1.56,2.95
a04,0.1,0.1,1.08,0.0,1.22,0.82,2.04
a05,1.17,1.17,2.29,1.22,0.0,1.03,1.1
a06,0.72,0.72,1.56,0.82,1.03,0.0,1.39
a07,1.96,1.96,2.95,2.04,1.1,1.39,0.0


# LOOPING

In [8]:
df = df_completo.copy()
df

,a01,a02,a03,a04,a05,a06,a07
a01,0.0,0.0,1.12,0.1,1.17,0.72,1.96
a02,0.0,0.0,1.12,0.1,1.17,0.72,1.96
a03,1.12,1.12,0.0,1.08,2.29,1.56,2.95
a04,0.1,0.1,1.08,0.0,1.22,0.82,2.04
a05,1.17,1.17,2.29,1.22,0.0,1.03,1.1
a06,0.72,0.72,1.56,0.82,1.03,0.0,1.39
a07,1.96,1.96,2.95,2.04,1.1,1.39,0.0


In [9]:
writer = pd.ExcelWriter('distancia_ligacao_completa.xlsx', engine='xlsxwriter')

#Tirando as diagonais e repetidos do lado superior direito
dff = df.copy()

lista = []
for coluna in dff.columns:
    for linha in dff.index:
        if coluna == linha:
            dff[coluna][linha] = ''

        if sorted(list([linha, coluna])) in lista:
            dff[coluna][linha] = ''
        else:
            lista.append(sorted(list([linha, coluna]))) 

dff.to_excel(writer, sheet_name=str(0))

In [10]:
cofenetica = pd.DataFrame(columns = ['Passo','Junção','Nivel'])

count = 0
#count < 5
#len(df.columns) > 2:
while len(df.columns) > 1:
    count += 1 
    
    #Excluindo diagonal
    for coluna in df.columns:
        for linha in df.index:
            if coluna == linha:
                df[coluna][linha] = np.nan

    #Encontrando menor valor na matrix
    menor_valor = 100

    for coluna in df.columns:
        for linha in df.index:
            if df[coluna][linha] == '':
                continue
            elif df[coluna][linha] < menor_valor:
                menor_valor = df[coluna][linha]
                ind_coluna = coluna
                ind_linha = linha

    juntar = ','.join(sorted(list([ind_coluna, ind_linha])))
    separar = juntar.split(',')
    
    juntar2 = ' - '.join(list([ind_coluna, ind_linha]))
    cofenetica.loc[len(cofenetica)] = [count, juntar2, menor_valor]

    #obs: 1 coluna, 0 linha
    df.drop(ind_coluna, axis=1, inplace=True)
    df.drop(ind_coluna, axis=0, inplace=True)

    df.drop(ind_linha, axis=0, inplace=True)
    df.drop(ind_linha, axis=1, inplace=True)

    df[juntar] = ''
    df.loc[juntar] = ''

    df.sort_index(axis=1, inplace=True)
    df.sort_index(axis=0, inplace=True)

    #Fazendo reformulação da matrix
    for coluna in df.columns:
        for linha in df.index:

            separar_coluna = coluna.split(',')
            separar_linha = linha.split(',')
            
            #Calculando distancia de mais de 2 individuo
            if (len(separar_coluna) > 1) and (len(separar_linha) > 1):

                lista = []
                for i in separar_coluna:
                    combinacoes = list(itertools.product([i], list(separar_linha)))

                    for comb in combinacoes:
                        colunazinha = list(comb)[0]
                        linhazinha = list(comb)[1]
                        valor = df_completo[colunazinha][linhazinha]
                        lista.append(valor)

                media = max(lista)
                df[coluna][linha] = round(media, 2)
            
            #Calculando distancia de 3 individuos
            elif len(separar_coluna) > 1:
                lista = []
                for i in separar_coluna:
                    lista.append(df_completo[i][linha])

                media = max(lista)
                df[coluna][linha] = round(media, 2)

            elif len(separar_linha) > 1:
                lista = []
                for i in separar_linha:
                    lista.append(df_completo[coluna][i])

                media = max(lista)
                df[coluna][linha] = round(media, 2)
            
            #Calculando distancia de 2 individuos
            else:
                df[coluna][linha] = df_completo[coluna][linha]

                
    #Tirando as diagonais e repetidos do lado superior direito
    dff = df.copy()
    
    lista = []
    for coluna in dff.columns:
        for linha in dff.index:
            if coluna == linha:
                dff[coluna][linha] = ''

            if sorted(list([linha, coluna])) in lista:
                dff[coluna][linha] = ''
            else:
                lista.append(sorted(list([linha, coluna])))            
                
                
    
    #display(df)
    dff.to_excel(writer, sheet_name=str(count))
    
#writer.save()

In [11]:
cofenetica

cofenetica.to_excel(writer, sheet_name='tabela cofenetica')

In [12]:
dfc = pd.DataFrame(index=fonte.index, columns = list(fonte.index))

for coluna in dfc.columns:
    for linha in dfc.index:
        for i in range(len(cofenetica)-1, -1, -1):
            if (coluna in cofenetica['Junção'][i]) and (linha in cofenetica['Junção'][i]):
                dfc[coluna][linha] = cofenetica['Nivel'][i]
                
display(df_completo)                
display(dfc)

,a01,a02,a03,a04,a05,a06,a07
a01,0.0,0.0,1.12,0.1,1.17,0.72,1.96
a02,0.0,0.0,1.12,0.1,1.17,0.72,1.96
a03,1.12,1.12,0.0,1.08,2.29,1.56,2.95
a04,0.1,0.1,1.08,0.0,1.22,0.82,2.04
a05,1.17,1.17,2.29,1.22,0.0,1.03,1.1
a06,0.72,0.72,1.56,0.82,1.03,0.0,1.39
a07,1.96,1.96,2.95,2.04,1.1,1.39,0.0


,a01,a02,a03,a04,a05,a06,a07
a01,0.0,0.0,1.56,0.1,2.95,0.82,2.95
a02,0.0,0.0,1.56,0.1,2.95,0.82,2.95
a03,1.56,1.56,1.56,1.56,2.95,1.56,2.95
a04,0.1,0.1,1.56,0.1,2.95,0.82,2.95
a05,2.95,2.95,2.95,2.95,1.1,2.95,1.1
a06,0.82,0.82,1.56,0.82,2.95,0.82,2.95
a07,2.95,2.95,2.95,2.95,1.1,2.95,1.1


In [13]:
lista = []
for coluna in df_completo.columns:
    for linha in df_completo.index:
        if coluna == linha:
            df_completo[coluna][linha] = ''

        if sorted(list([linha, coluna])) in lista:
            df_completo[coluna][linha] = ''
        else:
            lista.append(sorted(list([linha, coluna])))
            
display(df_completo)

lista = []
for coluna in dfc.columns:
    for linha in dfc.index:
        if coluna == linha:
            dfc[coluna][linha] = ''

        if sorted(list([linha, coluna])) in lista:
            dfc[coluna][linha] = ''
        else:
            lista.append(sorted(list([linha, coluna])))
            
display(dfc)

dfc.to_excel(writer, sheet_name='matrix cofenetica')

,a01,a02,a03,a04,a05,a06,a07
a01,,,,,,,
a02,0.0,,,,,,
a03,1.12,1.12,,,,,
a04,0.1,0.1,1.08,,,,
a05,1.17,1.17,2.29,1.22,,,
a06,0.72,0.72,1.56,0.82,1.03,,
a07,1.96,1.96,2.95,2.04,1.1,1.39,


,a01,a02,a03,a04,a05,a06,a07
a01,,,,,,,
a02,0.0,,,,,,
a03,1.56,1.56,,,,,
a04,0.1,0.1,1.56,,,,
a05,2.95,2.95,2.95,2.95,,,
a06,0.82,0.82,1.56,0.82,2.95,,
a07,2.95,2.95,2.95,2.95,1.1,2.95,


In [14]:
cor = pd.DataFrame(columns = ['S','C'])

for coluna in dfc.columns:
    for linha in dfc.index:
        if dfc[coluna][linha] != '':
            cor.loc[','.join(list([coluna, linha]))] = [df_completo[coluna][linha], dfc[coluna][linha]]
            
cor

cor.to_excel(writer, sheet_name='pareamento correlação')

In [15]:
cor['S'].corr(cor['C'])

0.7913114830488623

In [16]:
writer.save()